# Contents

### 1. Import dataframe
### 2. Address PII data in the dataframe
### 3. Regional segmentation
### 4. Exclusion flag for low-activity customers
### 5. Customer profiling
### 6. Visualization for distribution of profiles
### 7. Aggregation of customer profiles  
### 8. Analysis by regions
### 9. Visualizations

In [3]:
# Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [4]:
path = r'C:\Users\kshifera\Desktop\CF_Course\Immersion\Achievement_4\Instacart Basket Analysis'

### 1. Import dataframe

In [5]:
# import dataframe
df = pd.read_pickle(os.path.join(path, '02 Data' , 'Prepared Data' , 'df_9_1.pkl'))

In [6]:
# check
df.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,frequency_flag,first_name,surname,gender,state,age,date_joined,number_of_dependents,family_status,income
0,2539329,1,1,2,8,0.0,196,1,0,Soda,...,Regular customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Regular customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Regular customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Regular customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Regular customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


In [7]:
# check
df.shape

(32404859, 33)

### 2. Address PII data in the dataframe

   #### The dataframe contains what are usually termed as PII data which can be traced back to a particular person. From these dataframe, columns containing customer's first_name and surname have been excluded to ensure data privacy.

In [8]:
# drop columns 'first_name' & 'surname'
df = df.drop(columns = ['first_name', 'surname'])

In [9]:
# check
df.shape

(32404859, 31)

### 3. Regional segmentation

#### 3.1 Create region column

In [10]:
# create a 'region' column based on 'state' column
result = []

for value in df["state"]:
    if value in ['Maine', 'New Hampshire' , 'Vermont' , 'Massachusetts' , 'Rhode Island' , 'Connecticut' , 'New York' , 'Pennsylvania' , 'New Jersey']:
        result.append("Northeast")
    elif value in ['Wisconsin' , 'Michigan' , 'Illinois' , 'Indiana' , 'Ohio' , 'North Dakota' , 'South Dakota' , 'Nebraska' , 'Kansas' , 'Minnesota' , 'Iowa' , 'Missouri']:
        result.append("Midwest")
    elif value in ['Delaware' , 'Maryland' , 'District of Columbia' , 'Virginia' , 'West Virginia' , 'North Carolina' , 'South Carolina' , 'Georgia' , 'Florida' , 'Kentucky' , 'Tennessee' , 'Mississippi' , 'Alabama' , 'Oklahoma' , 'Texas' , 'Arkansas' , 'Louisiana']:
        result.append("South")
    else:
        result.append("West")


In [11]:
df['region'] = result

In [12]:
# check
df.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,median_order,frequency_flag,gender,state,age,date_joined,number_of_dependents,family_status,income,region
0,2539329,1,1,2,8,0.0,196,1,0,Soda,...,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South


#### 3.2 Spending habits across the regions

In [13]:
# we create a table that holds the absolute numbers of customers in each region 
crosstab = pd.crosstab(df['region'], df['spending_flag'], dropna = False)

In [14]:
crosstab

spending_flag,High spender,Low spender
region,,
Midwest,155975,7441350
Northeast,108225,5614511
South,209691,10582194
West,160354,8132559


In [15]:
# crosstab that displays spending habit as percentage of region total
crosstab_regional = pd.crosstab(df['region'], df['spending_flag'], normalize = 'index')

In [16]:
crosstab_regional

spending_flag,High spender,Low spender
region,,
Midwest,0.020530,0.979470
Northeast,0.018911,0.981089
South,0.019430,0.980570
West,0.019336,0.980664


In [17]:
#crosstab that displays proportion of spending  habit of each spender as percentage of total spending habit at Instacart
crosstab_spenders = pd.crosstab(df['region'], df['spending_flag'], normalize = 'columns')

In [18]:
crosstab_spenders

spending_flag,High spender,Low spender
region,,
Midwest,0.245922,0.234221
Northeast,0.170636,0.176720
South,0.330615,0.333081
West,0.252827,0.255977


The result indicate that while the proportion of high spenders to low spenders in each region is roughly the same, 98% to 2%, there are differences in the regional proportion as of the total when viewed for each category of spending. 33 % of the high spenders are in the South region, with West, Midwest and Northeast taking the next spots in that order. The same pattern holds for the low spending category as well. 

#### 3.3. distribution of brand loyalty

In [19]:
# brand loyalty across regions
loy_reg = pd.crosstab(df['region'], df['loyalty_flag'], normalize = 'columns')
loy_reg

loyalty_flag,Loyal customer,New customer,Regular customer
region,,,
Midwest,0.230820,0.235838,0.236256
Northeast,0.179091,0.176203,0.175145
South,0.331176,0.332225,0.334554
West,0.258913,0.255734,0.254045


### 4. Exclusion flag for low-activity customers

In [20]:
# Add a column holding exclusion flag
df.loc[df['order_number'] >= 5, 'exclusion_flag'] = 'high activity'

In [21]:
df.loc[df['order_number'] < 5, 'exclusion_flag'] = 'low activity'

In [22]:
# frequency check
df['exclusion_flag'].value_counts(dropna = False)

exclusion_flag
high activity    24414877
low activity      7989982
Name: count, dtype: int64

#### 4.1. ordering habits based on region

In [23]:
# activity by region
ord_reg = pd.crosstab(df['region'], df['exclusion_flag'], normalize = 'index')
ord_reg

exclusion_flag,high activity,low activity
region,,
Midwest,0.751811,0.248189
Northeast,0.754057,0.245943
South,0.753259,0.246741
West,0.754712,0.245288


Irrespective of region, 3 in 4 customers order atleast 5 times 

#### 4.2. ordering habits based on loyalty

In [24]:
# ordering habit and loyalty
ord_hab = pd.crosstab(df['loyalty_flag'], df['exclusion_flag'], normalize = 'index')
ord_hab

exclusion_flag,high activity,low activity
loyalty_flag,,
Loyal customer,0.929153,0.070847
New customer,0.338698,0.661302
Regular customer,0.802717,0.197283


 The vast majority of loyal customers or regular customers are high activity customers, while it is, understandably (given that they are new) the opposite for the new customers

In [25]:
# dataframe with 'high activity' customers
df_2 = df[df['exclusion_flag'] =='high activity']

In [26]:
df_2.shape

(24414877, 33)

In [30]:
# export this dataframe for the next part of the script
df_2.to_pickle(os.path.join(path, '02 Data' , 'Prepared Data' , 'df_chunk1.pkl' ))

customer profiling continues on next chunk...